In [1]:
import sys
import numpy as np
import tensorflow as tf

#no GPU since Apple Silicon have problem with mfcc
tf.config.set_visible_devices([], 'GPU')

import pydub
from pathlib import Path
sys.path.append("..")
from DeepFeature_DataSet.Test_DS_Factory_Siri import Test_DS_Factory_Siri

In [2]:
input_path = Path("../../DataFolder/Prosody_2024/Siri/")
output_path = Path("../../DataFolder/Prosody_2024/Siri_Frame_MFCs/")
number_of_actor= [1,2,4,5]

In [3]:
# All folder contain same amount of files with same name siriv1 ... siriv5

In [4]:
# Load one file
p = input_path / "siriV1"
list_of_files = list(p.glob("*.m4a"))

In [5]:
# Process all files
for file in list_of_files:
    all_actor = [file.parent.parent / f"SiriV{actor}" / file.name for actor in number_of_actor]
    frames = []
    for actor in all_actor:
        audio = pydub.AudioSegment.from_file(actor)
        audio = audio.set_frame_rate(16000)
        array = tf.convert_to_tensor(np.array(audio.get_array_of_samples()))
        mfc = Test_DS_Factory_Siri.get_mfcc(array)
        frames.append(mfc)

    # Find the maximum length of the tensors in frames in the time dimension (axis 1)
    max_len = max([frame.shape[0] for frame in frames])

    # Pad each tensor in frames to the maximum length in the time dimension
    padded_frames = [tf.pad(frame, paddings=[[0, max_len - frame.shape[0]], [0, 0]], constant_values=-1) for frame in frames]

    # Stack the padded tensors into a single tensor
    stacked_frames = tf.stack(padded_frames)
    # Write to file using tensorflow serialization
    tf.io.write_file(str(output_path / f"{file.stem}.tfs"), tf.io.serialize_tensor(stacked_frames))
    

I0000 00:00:1715288773.386264 1761939 service.cc:145] XLA service 0x600000865800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715288773.386343 1761939 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1715288773.443235 1761939 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [6]:
student_path = Path("../../DataFolder/Prosody_2024/ProsodyAudio24/recordings_wav")
student_mfc = student_path.parent.parent / "Student_MFCs"
student_mfc.mkdir(parents=True, exist_ok=True)

In [7]:
for stu_p in student_path.glob("**/*.wav"):
    audio = pydub.AudioSegment.from_file(stu_p)
    audio = audio.set_frame_rate(16000)
    array = tf.convert_to_tensor(np.array(audio.get_array_of_samples()))
    mfc = Test_DS_Factory_Siri.get_mfcc(array)[tf.newaxis, ...]
    save_path = (student_mfc / stu_p.relative_to(student_path)).parent / f"{stu_p.stem}.tfs"
    tf.io.write_file(str(save_path), tf.io.serialize_tensor(mfc))

In [8]:
save_path

PosixPath('../../DataFolder/Prosody_2024/Student_MFCs/b265dbbd-f353-446e-bc09-0a631e07e4f9/4/33004/1714680103497.tfs')